In [1]:
import keras
keras.__version__

Using TensorFlow backend.


'2.3.1'

#### 作业1: 多输入模型7.1.2

In [2]:
from keras.models import Model
from keras import layers
from keras import Input

In [4]:
# 多输入单输出
text_vocabulary_size = 10000   #参考文本最大长度
question_vocabulary_size = 10000   #问题文本最大长度
answer_vocabulary_size = 500   #答案个数

text_input = Input(shape=(None,),dtype='int32',name='text')
embedded_text = layers.Embedding(text_vocabulary_size,64)(text_input)  #将输入嵌入长度为64的词向量
encoded_text = layers.LSTM(32)(embedded_text)

question_input = Input(shape=(None,),dtype='int32',name='question')
embedded_question = layers.Embedding(question_vocabulary_size,32)(question_input)
encoded_question = layers.LSTM(16)(embedded_question)

concatenated = layers.concatenate([encoded_text,encoded_question],axis=-1)  #将编码后的问题和文本拼接起来
answer = layers.Dense(answer_vocabulary_size,activation='softmax')(concatenated)

# 将输入与输出放入Model中
model = Model([text_input,question_input],answer)
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['acc'])

In [5]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text (InputLayer)               (None, None)         0                                            
__________________________________________________________________________________________________
question (InputLayer)           (None, None)         0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, None, 64)     640000      text[0][0]                       
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, None, 32)     320000      question[0][0]                   
____________________________________________________________________________________________

In [6]:
import numpy as np

num_samples = 1000  #1000个问题
max_length = 100  #每个问题的最大长度

text = np.random.randint(0,text_vocabulary_size,size=(num_samples,max_length))  #随机生成参考文本
question= np.random.randint(0,question_vocabulary_size,size=(num_samples,max_length))  #随机生成参考问题
answers = np.random.randint(0,answer_vocabulary_size,num_samples)  #随机生成答案
answers = keras.utils.to_categorical(answers,answer_vocabulary_size)  #将答案独热编码

model.fit([text,question],answers,epochs=10,batch_size=128)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
1000/1000 [==============================] - 2s 2ms/step - loss: 6.2145 - acc: 0.0010
Epoch 2/10
1000/1000 [==============================] - 1s 564us/step - loss: 6.1952 - acc: 0.0260
Epoch 3/10
1000/1000 [==============================] - 1s 549us/step - loss: 6.1273 - acc: 0.0060
Epoch 4/10
1000/1000 [==============================] - 1s 565us/step - loss: 6.0516 - acc: 0.0060
Epoch 5/10
1000/1000 [==============================] - 1s 566us/step - loss: 5.9913 - acc: 0.0090
Epoch 6/10
1000/1000 [==============================] - 1s 557us/step - loss: 5.8924 - acc: 0.0150
Epoch 7/10
1000/1000 [==============================] - 1s 561us/step - loss: 5.7856 - acc: 0.0230
Epoch 8/10
1000/1000 [==============================] - 1s 558us/step - loss: 5.7043 - acc: 0.0340
Epoch 9/10
1000/1000 [==============================] - 1s 562us/step - loss: 5.6263 - acc: 0.0330
Epoch 10/10
1000/1000 [==============================] - 1s 556

#### 多输出模型7.1.3

In [8]:
#三输出模型(年龄、收入、性别)
from keras import layers
from keras import Input
from keras.models import Model

vocabulary_size = 50000  #社交媒体发帖最大词数
num_income_groups = 10  #收入分组数

posts_input = Input(shape=(None,),dtype='int32',name='posts')
embedded_posts = layers.Embedding(vocabulary_size,256)(posts_input)
x = layers.Conv1D(128,5,activation='relu')(embedded_posts)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256,5,activation='relu')(x)
x = layers.Conv1D(256,5,activation='relu')(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256,5,activation='relu')(x)
x = layers.Conv1D(256,5,activation='relu')(x)
x = layers.GlobalMaxPool1D()(x)
x = layers.Dense(128,activation='relu')(x)

age_prediction = layers.Dense(1,name='age')(x)
income_prediction = layers.Dense(num_income_groups,activation='softmax',name='income')(x)
gender_prediction = layers.Dense(1,activation='sigmoid',name='gender')(x)

model = Model(posts_input,[age_prediction,income_prediction,gender_prediction])

In [9]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
posts (InputLayer)              (None, None)         0                                            
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, None, 256)    12800000    posts[0][0]                      
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, None, 128)    163968      embedding_5[0][0]                
__________________________________________________________________________________________________
max_pooling1d_1 (MaxPooling1D)  (None, None, 128)    0           conv1d_1[0][0]                   
____________________________________________________________________________________________

In [10]:
model.compile(optimizer='rmsprop',
              loss=['mse','categorical_crossentropy' ,'binary_crossentropy'],
              loss_weights = [0.25,1.0,10.])

In [18]:
import numpy as np

num_samples = 1000  #1000个发帖
max_length = 300  #每个帖子的最大长度

posts = np.random.randint(0,vocabulary_size,size = (num_samples,max_length))

age_targets = np.random.random(num_samples)*80+10
income_targets = np.random.randint(0,num_income_groups,num_samples)
income_targets = keras.utils.to_categorical(income_targets,num_income_groups)
gender_targets = np.random.randint(0,2,num_samples)

In [19]:
model.fit(posts,[age_targets,income_targets,gender_targets],epochs=10,batch_size=64)

Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/10
1000/1000 [==============================] - 7s 7ms/step - loss: 387.4078 - age_loss: 1451.5447 - income_loss: 3.9334 - gender_loss: 1.6006
Epoch 2/10
1000/1000 [==============================] - 6s 6ms/step - loss: 173.5352 - age_loss: 645.9570 - income_loss: 2.7508 - gender_loss: 0.9198
Epoch 3/10
1000/1000 [==============================] - 6s 6ms/step - loss: 169.0385 - age_loss: 624.5934 - income_loss: 2.3936 - gender_loss: 0.8054
Epoch 4/10
1000/1000 [==============================] - 6s 6ms/step - loss: 127.7919 - age_loss: 460.1621 - income_loss: 2.3948 - gender_loss: 0.9512
Epoch 5/10
1000/1000 [==============================] - 6s 6ms/step - loss: 91.9249 - age_loss: 323.2721 - income_loss: 2.3776 - gender_loss: 0.7890
Epoch 6/10
1000/1000 [==============================] - 6s 6ms/step - loss: 91.4348 - age_loss: 309.5790 - income_loss: 2.4463 - gender_loss: 1.0097
Epoch 7/10
1000/1000 [=

#### 作业2:对例6.2.3(IMDB的RNN模型)引入回调函数

In [26]:
#读取数据
from keras.datasets import imdb
from keras.preprocessing import sequence

max_features = 10000  #最大特征数,即该训练集最常见的前10000个单词
maxlen = 20  #每条评论最大长度为20，超过将被截断

#将数据加载为整数列表
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

#整数列表填充处理:即超长截断、不足则前补0，默认从尾部截取，得到等长二维整数张量(samples,maxlen)
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

In [27]:
from keras.models import Sequential
from keras.layers import Dense,LSTM,Embedding

model = Sequential()
model.add(Embedding(max_features, 32))
model.add(LSTM(32))
model.add(Dense(1, activation='sigmoid'))

#回调函数列表
callbacks_list = [
    keras.callbacks.EarlyStopping(
        monitor='val_acc',  #监控模型的验证精度
        patience=2,  #验证精度多余2轮的时间(即3轮)内不改善，则终止模型
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.1,# 如果符合条件，学习速率变为原来的0.5
        patience = 5  #验证损失5轮内不改善，则终止模型
    )
]

In [28]:
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])
history = model.fit(x_train, y_train,
                    epochs=20,
                    batch_size=128,
                    callbacks=callbacks_list,  #引入回调函数
                    validation_split=0.2)

Train on 20000 samples, validate on 5000 samples
Epoch 1/20
20000/20000 [==============================] - 3s 166us/step - loss: 0.5878 - acc: 0.6888 - val_loss: 0.5252 - val_acc: 0.7304
Epoch 2/20
20000/20000 [==============================] - 2s 103us/step - loss: 0.4459 - acc: 0.7915 - val_loss: 0.5017 - val_acc: 0.7490
Epoch 3/20
20000/20000 [==============================] - 2s 103us/step - loss: 0.3968 - acc: 0.8207 - val_loss: 0.4930 - val_acc: 0.7548
Epoch 4/20
20000/20000 [==============================] - 2s 103us/step - loss: 0.3666 - acc: 0.8378 - val_loss: 0.5128 - val_acc: 0.7522
Epoch 5/20
20000/20000 [==============================] - 2s 102us/step - loss: 0.3456 - acc: 0.8486 - val_loss: 0.5330 - val_acc: 0.7478


#### 作业3:用Tensorboard查看(IMDB的RNN模型)运行过程的可视化结果

In [25]:
mkdir my_log_dir

In [31]:
from keras.models import Sequential
from keras.layers import Dense,LSTM,Embedding

model = Sequential()
model.add(Embedding(max_features, 32))
model.add(LSTM(32))
model.add(Dense(1, activation='sigmoid'))

callbacks = [
    keras.callbacks.TensorBoard(
        log_dir='my_log_dir',
        histogram_freq = 1,
    )
]

In [32]:
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])
history = model.fit(x_train, y_train,
                    epochs=20,
                    batch_size=128,
                    callbacks=callbacks,
                    validation_split=0.2)

Train on 20000 samples, validate on 5000 samples
Epoch 1/20
20000/20000 [==============================] - 3s 172us/step - loss: 0.5794 - acc: 0.6921 - val_loss: 0.5407 - val_acc: 0.7236
Epoch 2/20
20000/20000 [==============================] - 2s 102us/step - loss: 0.4434 - acc: 0.7944 - val_loss: 0.4935 - val_acc: 0.7518
Epoch 3/20
20000/20000 [==============================] - 2s 101us/step - loss: 0.3959 - acc: 0.8226 - val_loss: 0.5008 - val_acc: 0.7502
Epoch 4/20
20000/20000 [==============================] - 2s 101us/step - loss: 0.3673 - acc: 0.8352 - val_loss: 0.4930 - val_acc: 0.7504
Epoch 5/20
20000/20000 [==============================] - 2s 107us/step - loss: 0.3466 - acc: 0.8500 - val_loss: 0.5110 - val_acc: 0.7470
Epoch 6/20
20000/20000 [==============================] - 2s 108us/step - loss: 0.3280 - acc: 0.8590 - val_loss: 0.5375 - val_acc: 0.7352
Epoch 7/20
20000/20000 [==============================] - 2s 102us/step - loss: 0.3104 - acc: 0.8695 - val_loss: 0.5336 - v